<a href="https://colab.research.google.com/github/nitinit/ai-ml/blob/main/Intent_Slot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import BertTokenizerFast, BertModel
from collections import defaultdict
import json

# --------- 1. Expanded Training Data ---------
# Your prompts split into words with slot labels and intent

data = [
    # add_expense
    {"words": ["Add", "expense", "of", "$20", "to", "group", "Travel", "Friends"],
     "slots": ["O", "O", "O", "B-amount", "O", "O", "B-group", "I-group"], "intent": "add_expense"},
    {"words": ["Add", "rent", "of", "$1200", "to", "House", "Bills", "group"],
     "slots": ["O", "O", "O", "B-amount", "O", "B-group", "I-group", "O"], "intent": "add_expense"},
    {"words": ["Add", "$95", "groceries", "to", "House", "Bills", "group"],
     "slots": ["O", "B-amount", "O", "O", "B-group", "I-group", "O"], "intent": "add_expense"},
    {"words": ["Add", "a", "$350", "software", "charge", "to", "Startup", "Budget", "group"],
     "slots": ["O", "O", "B-amount", "O", "O", "O", "B-group", "I-group", "O"], "intent": "add_expense"},
    {"words": ["Log", "$60", "Spotify", "family", "plan", "to", "Monthly", "Subscriptions"],
     "slots": ["O", "B-amount", "B-service", "I-service", "I-service", "O", "B-group", "I-group"], "intent": "add_expense"},
    {"words": ["Add", "equipment", "bill", "of", "$480", "to", "Band", "Practice", "group"],
     "slots": ["O", "O", "O", "O", "B-amount", "O", "B-group", "I-group", "O"], "intent": "add_expense"},
    {"words": ["Add", "a", "dinner", "bill", "of", "$240", "to", "Tahoe", "trip", "group"],
     "slots": ["O", "O", "O", "O", "O", "B-amount", "O", "B-group", "I-group", "O"], "intent": "add_expense"},

    # add_member
    {"words": ["Add", "Sarah", "to", "Startup", "Budget", "group"],
     "slots": ["O", "B-person", "O", "B-group", "I-group", "O"], "intent": "add_member"},
    {"words": ["Add", "Charlie", "to", "group", "Travel", "Friends"],
     "slots": ["O", "B-person", "O", "O", "B-group", "I-group"], "intent": "add_member"},
    {"words": ["Add", "Mike", "to", "the", "House", "Bills", "group"],
     "slots": ["O", "B-person", "O", "O", "B-group", "I-group", "O"], "intent": "add_member"},
    {"words": ["Add", "multiple", "people", ":", "Josh", ",", "Amy", ",", "and", "Chris", "to", "Tahoe", "Trip"],
     "slots": ["O", "O", "O", "O", "B-person", "O", "B-person", "O", "O", "B-person", "O", "B-group", "I-group"], "intent": "add_member"},

    # check_balance
    {"words": ["What's", "my", "share", "of", "rent", "in", "House", "Bills", "?"],
     "slots": ["O", "O", "O", "O", "O", "O", "B-group", "I-group", "O"], "intent": "check_balance"},
    {"words": ["Do", "I", "owe", "the", "group", "money", "?"],
     "slots": ["O", "O", "O", "O", "O", "O", "O"], "intent": "check_balance"},
    {"words": ["What's", "my", "total", "due", "across", "all", "groups", "?"],
     "slots": ["O", "O", "O", "O", "O", "O", "O", "O"], "intent": "check_balance"},
    {"words": ["Do", "I", "owe", "anyone", "in", "the", "Startup", "Budget", "group", "?"],
     "slots": ["O", "O", "O", "O", "O", "O", "B-group", "I-group", "O", "O"], "intent": "check_balance"},
    {"words": ["How", "much", "do", "I", "owe", "in", "the", "group", "for", "my", "band", "practice", "?"],
     "slots": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-group", "I-group", "O"], "intent": "check_balance"},
    {"words": ["Am", "I", "owed", "money", "from", "groups", "?"],
     "slots": ["O", "O", "O", "O", "O", "O", "O"], "intent": "check_balance"},

    # create_group
    {"words": ["Create", "a", "group", "called", "Travel", "Friends", "with", "Bob", "and", "Alice"],
     "slots": ["O", "O", "O", "O", "B-group", "I-group", "O", "B-person", "O", "B-person"], "intent": "create_group"},
    {"words": ["Create", "a", "new", "group", "Monthly", "Subscriptions", "with", "Alex", "and", "Ben"],
     "slots": ["O", "O", "O", "O", "B-group", "I-group", "O", "B-person", "O", "B-person"], "intent": "create_group"},
    {"words": ["Start", "a", "group", "called", "Startup", "Budget", "with", "John", "and", "Emma"],
     "slots": ["O", "O", "O", "O", "B-group", "I-group", "O", "B-person", "O", "B-person"], "intent": "create_group"},
    {"words": ["Make", "a", "group", "for", "House", "Bills", "with", "Lisa", "and", "Tom"],
     "slots": ["O", "O", "O", "O", "B-group", "I-group", "O", "B-person", "O", "B-person"], "intent": "create_group"},

    # group_summary
    {"words": ["Show", "breakdown", "for", "Weekend", "Warriors", "group"],
     "slots": ["O", "O", "O", "B-group", "I-group", "O"], "intent": "group_summary"},
    {"words": ["Show", "me", "a", "summary", "of", "the", "group", "for", "my", "band", "practice"],
     "slots": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "B-group", "I-group"], "intent": "group_summary"},
    {"words": ["Show", "full", "activity", "of", "Startup", "Budget", "group"],
     "slots": ["O", "O", "O", "O", "B-group", "I-group", "O"], "intent": "group_summary"},
    {"words": ["Summary", "of", "all", "groups", "with", "pending", "balances"],
     "slots": ["O", "O", "O", "O", "O", "O", "O"], "intent": "group_summary"},

    # hide_groups
    {"words": ["Hide", "all", "my", "groups", "with", "zero", "balance"],
     "slots": ["O", "O", "O", "O", "O", "O", "O"], "intent": "hide_groups"},
]


In [2]:
# --------- 2. Label Mappings ---------
intents = list(set(item["intent"] for item in data))
intent2id = {intent: idx for idx, intent in enumerate(intents)}
id2intent = {v: k for k, v in intent2id.items()}

slot_labels = set()
for item in data:
    slot_labels.update(item["slots"])
slot_labels = sorted(slot_labels)
slot_label2id = {label: idx for idx, label in enumerate(slot_labels)}
id2slot_label = {v: k for k, v in slot_label2id.items()}

print("Intents:", intent2id)
print("Slots:", slot_label2id)

Intents: {'check_balance': 0, 'create_group': 1, 'add_member': 2, 'hide_groups': 3, 'group_summary': 4, 'add_expense': 5}
Slots: {'B-amount': 0, 'B-group': 1, 'B-person': 2, 'B-service': 3, 'I-group': 4, 'I-service': 5, 'O': 6}


In [3]:
# --------- 3. Tokenizer ---------
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
# --------- 4. Dataset ---------
def tokenize_and_align_labels(texts, slot_labels_list):
    encodings = tokenizer(texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
    all_labels = []
    for i, labels in enumerate(slot_labels_list):
        word_ids = encodings.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(slot_label2id.get(labels[word_idx], slot_label2id["O"]))
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        all_labels.append(label_ids)
    return encodings, all_labels

class JointDataset(Dataset):
    def __init__(self, data):
        self.texts = [item["words"] for item in data]
        self.slots = [item["slots"] for item in data]
        self.intents = [intent2id[item["intent"]] for item in data]
        self.encodings, self.slot_labels = tokenize_and_align_labels(self.texts, self.slots)
    def __len__(self):
        return len(self.intents)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items() if key != "offset_mapping"}
        item['labels'] = torch.tensor(self.slot_labels[idx])
        item['intent_label'] = torch.tensor(self.intents[idx])
        return item

dataset = JointDataset(data)

In [5]:
# --------- 5. Model ---------
class JointBERT(nn.Module):
    def __init__(self, num_intents, num_slots):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        hidden_size = self.bert.config.hidden_size
        self.intent_classifier = nn.Linear(hidden_size, num_intents)
        self.slot_classifier = nn.Linear(hidden_size, num_slots)
    def forward(self, input_ids, attention_mask, labels=None, intent_label=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        pooled_output = outputs.pooler_output
        intent_logits = self.intent_classifier(pooled_output)
        slot_logits = self.slot_classifier(sequence_output)
        loss = None
        if labels is not None and intent_label is not None:
            loss_fct = nn.CrossEntropyLoss()
            slot_loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            intent_loss = loss_fct(intent_logits, intent_label)
            slot_loss = slot_loss_fct(slot_logits.view(-1, slot_logits.shape[-1]), labels.view(-1))
            loss = intent_loss + slot_loss
        return loss, intent_logits, slot_logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = JointBERT(num_intents=len(intent2id), num_slots=len(slot_label2id)).to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [7]:
# --------- 6. Training ---------
model.train()
for epoch in range(8):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k,v in batch.items()}
        optimizer.zero_grad()
        loss, _, _ = model(input_ids=batch["input_ids"],
                           attention_mask=batch["attention_mask"],
                           labels=batch["labels"],
                           intent_label=batch["intent_label"])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {total_loss/len(train_loader):.4f}")


Epoch 1 loss: 0.1919
Epoch 2 loss: 0.1384
Epoch 3 loss: 0.1112
Epoch 4 loss: 0.1023
Epoch 5 loss: 0.0745
Epoch 6 loss: 0.0603
Epoch 7 loss: 0.0514
Epoch 8 loss: 0.0478


In [8]:
# --------- 7. Prediction ---------
def predict(text):
    model.eval()
    words = text.split()
    inputs = tokenizer([words], is_split_into_words=True, return_tensors="pt").to(device)
    with torch.no_grad():
        _, intent_logits, slot_logits = model(inputs["input_ids"], inputs["attention_mask"])
        intent_id = torch.argmax(intent_logits, dim=1).item()
        intent = id2intent[intent_id]
        slot_pred = torch.argmax(slot_logits, dim=2).squeeze().tolist()
    word_ids = inputs.word_ids(batch_index=0)
    results = defaultdict(list)
    current_slot = None
    current_words = []
    for idx, word_idx in enumerate(word_ids):
        if word_idx is None:
            continue
        slot_label = id2slot_label[slot_pred[idx]]
        word = words[word_idx]
        if slot_label == "O":
            if current_slot:
                results[current_slot].append(" ".join(current_words))
                current_slot = None
                current_words = []
        elif slot_label.startswith("B-"):
            if current_slot:
                results[current_slot].append(" ".join(current_words))
            current_slot = slot_label[2:]
            current_words = [word]
        elif slot_label.startswith("I-") and current_slot == slot_label[2:]:
            current_words.append(word)
        else:
            if current_slot:
                results[current_slot].append(" ".join(current_words))
            current_slot = None
            current_words = []
    if current_slot:
        results[current_slot].append(" ".join(current_words))
    for k in results:
        if len(results[k]) == 1:
            results[k] = results[k][0]
    return intent, dict(results)

In [9]:
# --------- 8. Test ---------
test_sentences = [
    "Add expense of $20 to group Travel Friends",
    "Add Sarah to Startup Budget group",
    "What's my share of rent in House Bills?",
    "Create a group called Travel Friends with Bob and Alice",
    "Show me a summary of the group for my band practice",
    "Hide all my groups with zero balance"
]

for sent in test_sentences:
    intent, slots = predict(sent)
    print(f"\nInput: {sent}")
    print(f"Predicted intent: {intent}")
    print(f"Predicted slots:\n{json.dumps(slots, indent=2)}")


Input: Add expense of $20 to group Travel Friends
Predicted intent: add_expense
Predicted slots:
{
  "amount": "$20",
  "group": "Travel Friends"
}

Input: Add Sarah to Startup Budget group
Predicted intent: add_member
Predicted slots:
{
  "person": "Sarah",
  "group": "Startup Budget"
}

Input: What's my share of rent in House Bills?
Predicted intent: check_balance
Predicted slots:
{
  "group": "House Bills?"
}

Input: Create a group called Travel Friends with Bob and Alice
Predicted intent: create_group
Predicted slots:
{
  "group": "Travel Friends",
  "person": [
    "Bob",
    "Alice"
  ]
}

Input: Show me a summary of the group for my band practice
Predicted intent: group_summary
Predicted slots:
{
  "group": "band practice"
}

Input: Hide all my groups with zero balance
Predicted intent: hide_groups
Predicted slots:
{}
